In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## getting dataset
import os
from os import listdir

#########################
# configuration
#########################
data_path = 'drive/MyDrive/DS_Project_Datasets/Networkrail/Delay/'
file_most_delay = '2021_22_P12_all_delays.csv'

In [28]:
import dateutil.parser
data = pd.read_csv(data_path + file_most_delay, low_memory=False)
data['ORIGIN_DEPARTURE_DATE'] = data['ORIGIN_DEPARTURE_DATE'].apply(lambda s: dateutil.parser.parse(s).date().strftime('%Y-%m-%d'))
data = data[data['ORIGIN_DEPARTURE_DATE'] == '2022-02-18']

In [29]:
delay = data['PFPI_MINUTES'].sum()
trains = data['TRUST_TRAIN_ID'].count()
delay_per_train = delay/trains

delay_per_train

14.92563372995344

In [30]:
#get incident dict
incident_df = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident JPIP Category']]
incident_df = incident_df.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident JPIP Category": "INCIDENT_CATEGORY"})
incident_df['INCIDENT_CATEGORY'].unique()

incident_df = incident_df.drop(labels=[incident_df[incident_df['INCIDENT_REASON'].isnull()].index[0]], axis="index")

In [32]:
tmp = data 
drop_col = tmp.columns.tolist()
drop_col.remove('INCIDENT_REASON')
drop_col.remove('PFPI_MINUTES')
tmp = tmp.drop(drop_col, axis = 1)
tmp = pd.merge(tmp, incident_df, on="INCIDENT_REASON", how = 'left')

In [47]:
drop_col = tmp.columns.tolist()
drop_col.remove('INCIDENT_CATEGORY')
drop_col.remove('PFPI_MINUTES')
tmp = tmp.drop(drop_col, axis = 1)
grouped_tmp = tmp.groupby(by=['INCIDENT_CATEGORY']).apply(lambda s: pd.Series({ "minutes": s["PFPI_MINUTES"].sum(),  "count": s["PFPI_MINUTES"].count()})).sort_values('minutes', axis=0, ascending=False)
top_3 = grouped_tmp.head(3)
print(top_3.to_latex())
print('sum of all reasons', grouped_tmp['count'].sum())

top_3

\begin{tabular}{lrr}
\toprule
{} &    minutes &    count \\
INCIDENT\_CATEGORY                   &            &          \\
\midrule
Severe Weather, Autumn \& Structures &  294579.25 &  16308.0 \\
Operations                          &   34789.00 &   1962.0 \\
TOC Other                           &   11494.25 &    959.0 \\
\bottomrule
\end{tabular}

sum of all reasons 25129.0


,minutes,count
INCIDENT_CATEGORY,,
"Severe Weather, Autumn & Structures",294579.25,16308.0
Operations,34789.00,1962.0
TOC Other,11494.25,959.0
